
# **Top music hits based on spottify top 200 weekly charts**

Our question of research is - What makes a song a hit in a given country. We will try to predict if a given song given it's features will become a top hit.

In attempt to answer this question we will try to identify some features that might affect a song's popularity based on it's number of streams.

In [149]:
# !pip install plotly
# !pip install "jupyterlab>=3" "ipywidgets>=7.6"
# !pip install -U kaleido
# !pip install folium

In [150]:
# from google.colab import drive
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

import math
import random
from datetime import timedelta


from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)



## Data collection and processing

### Get top 200 charts from Spotify

After we realized that the top 200 track charts didnt have any API, we built a crawler that download the CSV file from each week from 2018 to current.

we downloaded date for all 60 countries that spotify have charts on but for this study case we will analize only Israel charts.

you can see the process on our [github project](https://github.com/ShakedZrihen/worldwide-song-hit-formula).

### Merge data and add extra features

After we finished with data collection, we fetched from Spotify API featueres for each song. we created a CSV that holds the uniqe songs with song features and merge it with the merged week csv we downloaded.

We got artists extra data (genre, gender, country) from both Spotify API and MusicBrainz API.

### TODO - Missing data
* Add extra data for each Artist like popularity and activity period.
* Add some data related to events over the weeks (for example COVID time, seasons, special events, etc)

In [151]:
charts_df = pd.read_csv('chartsData.csv')
charts_df['id'] = charts_df['URL'].apply(lambda x: x.replace('https://open.spotify.com/track/', ''))
charts_df.drop(['Unnamed: 0', 'URL'], axis=1, inplace=True)
charts_df.set_index('id', inplace=True)
charts_df.head()


,Artist,Position,Streams,Track Name,country,dates
id,,,,,,
3CYH422oy1cZNoo0GTG1TK,Red Velvet,1,205528,Psycho,Viet Nam,2019-12-27--2020-01-03
3CYH422oy1cZNoo0GTG1TK,Red Velvet,106,314994,Psycho,Indonesia,2020-11-13--2020-11-20
3CYH422oy1cZNoo0GTG1TK,Red Velvet,107,79162,Psycho,Thailand,2020-11-20--2020-11-27
3CYH422oy1cZNoo0GTG1TK,Red Velvet,82,64279,Psycho,Singapore,2020-07-31--2020-08-07
3CYH422oy1cZNoo0GTG1TK,Red Velvet,91,367569,Psycho,Indonesia,2020-10-09--2020-10-16


In [152]:
tracks_data_df = pd.read_csv('finalData/uniquSongsData.csv', index_col='id')
tracks_data_df.drop(['track_href', 'key', 'name'], axis=1, inplace=True)
tracks_data_df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,numeric_key,popularity,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,
3CYH422oy1cZNoo0GTG1TK,0.1930,0.664,210560.0,0.666,0.000000,0.0854,-2.203,1.0,8.0,76.0,0.0446,139.987,4.0,0.405
1U8bwOTAtnqlWMCn1mwYyj,0.9690,0.516,305786.0,0.446,0.000415,0.1280,-9.384,0.0,0.0,18.0,0.0345,88.803,4.0,0.144
1rgnBhdG2JDFTbYkYRZAku,0.6880,0.825,209755.0,0.593,0.000161,0.1700,-6.401,0.0,6.0,65.0,0.0988,98.078,4.0,0.540
22F7P7QnaVspdf5rKveBaS,0.8400,0.634,252953.0,0.476,0.000137,0.0790,-7.714,0.0,4.0,12.0,0.0393,184.061,4.0,0.425
6v3KW9xbzN5yKLt9YKDYA2,0.0392,0.759,190800.0,0.548,0.000000,0.0828,-6.049,0.0,9.0,79.0,0.0290,116.967,4.0,0.749


In [153]:
charts = charts_df.drop_duplicates()
tracks = tracks_data_df.drop_duplicates()

In [154]:
merged = pd.merge(charts, tracks, how='left', on='id')
merged_2 = merged.reset_index()
df = merged_2.set_index(['id', 'Artist', 'Track Name'])
df['dates'] = df['dates'].apply(lambda x: x.split('--')[0])
df['dates'] = df['dates'].astype('datetime64[ns]')
df.rename({'dates': 'week'}, axis=1, inplace=True)
df.head()

Position  Streams    country  \
id                     Artist     Track Name                                 
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho             1   205528   Viet Nam   
                                  Psycho           106   314994  Indonesia   
                                  Psycho           107    79162   Thailand   
                                  Psycho            82    64279  Singapore   
                                  Psycho            91   367569  Indonesia   

                                                   week  acousticness  \
id                     Artist     Track Name                            
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho     2019-12-27         0.193   
                                  Psycho     2020-11-13         0.193   
                                  Psycho     2020-11-20         0.193   
                                  Psycho     2020-07-31         0.193   
                                  Psycho     2020-10-09         0.193   

                                              danceability  duration_ms  \
id                     Artist     Track Name                              
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   

                                              energy  instrumentalness  \
id                     Artist     Track Name                             
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   

                                              liveness  loudness  mode  \
id                     Artist     Track Name                             
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   

                                              numeric_key  popularity  \
id                     Artist     Track Name                            
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   

                                              speechiness    tempo  \
id                     Artist     Track Name                         
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   

                                              time_signature  valence  
id                     Artist     Track Name                           
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho                 4.0    0.405  
                                  Psycho                 4.0    0.405  
                                  Psycho                 4.0    0.405  
                                  Psycho                 4.0

## Basic data set Exploration


In [155]:
df.shape

(2142130, 18)

There are 2,142,130 rows in this Data Frame.
There are no duplicates because they were dropped before merging.

Lets explore the features:

## About the features

|feature | description|
|--------|------------|
|**week**|The week of the chart.|
|**Country**|The country of specified week top 200 chart.|
|**Streams**|Number of streams in specified country at specified week.|
|**Position**|Position of track in 200 top chart of specified country at specified week.|
|**popularity**|The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past.|
|**acousticness**| A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.|
|**danceability**|Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.|
|**duration_ms**|The duration of the track in milliseconds.	|
|**energy**|Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. <br/>For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.|
|**instrumentalness**|Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. <br/>The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.	|
|**numeric_key**|The key the track is in. Integers map to pitches using standard Pitch Class notation|
|**liveness**|Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.|
|**loudness**|The overall loudness of a track in decibels (dB). <be/>Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks.<br/> Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.|
|**mode**|Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.|
|**speechiness**|Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. <br/>Values above 0.66 describe tracks that are probably made entirely of spoken words.<br/> Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.|
|**tempo**|The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.|
|**Time Signature**|An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).|

In [156]:
df.dtypes

Position                     int64
Streams                      int64
country                     object
week                datetime64[ns]
acousticness               float64
danceability               float64
duration_ms                float64
energy                     float64
instrumentalness           float64
liveness                   float64
loudness                   float64
mode                       float64
numeric_key                float64
popularity                 float64
speechiness                float64
tempo                      float64
time_signature             float64
valence                    float64
dtype: object

In [157]:
top10 = df.loc[df['Position'] <= 10]
top10.head()

Position  Streams   country  \
id                     Artist     Track Name                                
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho             1   205528  Viet Nam   
                                  Psycho             5    98150  Viet Nam   
                                  Psycho             5   359916  Malaysia   
                                  Psycho             1   152883  Viet Nam   
                                  Psycho             1   492642  Thailand   

                                                   week  acousticness  \
id                     Artist     Track Name                            
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho     2019-12-27         0.193   
                                  Psycho     2020-01-31         0.193   
                                  Psycho     2020-01-03         0.193   
                                  Psycho     2019-12-20         0.193   
                                  Psycho     2020-01-10         0.193   

                                              danceability  duration_ms  \
id                     Artist     Track Name                              
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   
                                  Psycho             0.664     210560.0   

                                              energy  instrumentalness  \
id                     Artist     Track Name                             
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   
                                  Psycho       0.666               0.0   

                                              liveness  loudness  mode  \
id                     Artist     Track Name                             
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   
                                  Psycho        0.0854    -2.203   1.0   

                                              numeric_key  popularity  \
id                     Artist     Track Name                            
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   
                                  Psycho              8.0        76.0   

                                              speechiness    tempo  \
id                     Artist     Track Name                         
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   
                                  Psycho           0.0446  139.987   

                                              time_signature  valence  
id                     Artist     Track Name                           
3CYH422oy1cZNoo0GTG1TK Red Velvet Psycho                 4.0    0.405  
                                  Psycho                 4.0    0.405  
                                  Psycho                 4.0    0.405  
                                  Psycho                 4.0    0.4

In [158]:
top10_by_country = top10.reset_index().drop(['id', 'Artist', 'Track Name', 'Position', 'Streams'], axis=1).set_index(['country', 'week'])
features = top10_by_country.columns.tolist()
features

['acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'mode',
 'numeric_key',
 'popularity',
 'speechiness',
 'tempo',
 'time_signature',
 'valence']

In [159]:
mean_features_dfs = [top10_by_country.reset_index()[['country', 'week', feature]].copy().groupby(['country', 'week']).mean().reset_index() for feature in features]

In [160]:
mean_features_dfs[1]


,country,week,danceability
0,Argentina,2018-03-09,0.7808
1,Argentina,2018-03-16,0.7747
2,Argentina,2018-03-23,0.7747
3,Argentina,2018-03-30,0.7751
4,Argentina,2018-04-06,0.7747
...,...,...,...
10715,Viet Nam,2021-04-23,0.6165
10716,Viet Nam,2021-04-30,0.6165
10717,Viet Nam,2021-05-07,0.6409
10718,Viet Nam,2021-05-14,0.6759


In [162]:
mean_df = mean_features_dfs[1]
fig = px.choropleth(mean_df, locations='country', locationmode='country names', color = np.log(mean_df['danceability']), hover_name = 'country', animation_frame=mean_df['week'].dt.strftime('%m-%Y'), title='Top10 mean Acousticness over weeks', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale = True)
fig.show()

In [ ]:
import plotly.express as px

fig = px.choropleth(locations=["CA", "TX", "NY"], locationmode="USA-states", color=[1,2,3], scope="usa")
fig.show()

In [ ]:
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()